In [4]:
import torch
import h5py
import diffusion_pde as dpde
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [5]:
root_dir = dpde.utils.get_repo_root()
data_dir = root_dir / "data" / "llg_temp"
data_path = data_dir / "1000_150_64_16.h5"

In [8]:
with h5py.File(data_path, "r") as f:
    for key in f.keys():
        print(key)
    [print(f"{k}: {v}" for k, v in f.attrs.items())];

field
sequence
<generator object <genexpr> at 0x736487be49e0>
